In [1]:
from mesa.experimental import JupyterViz

from poc.simple import VECModel, VehicleAgent, VECStationAgent, VEC_STATION_COLORS, render_distance_chart


In [2]:
def agent_portrayal(agent):
    # Render vehicle agent
    if isinstance(agent, VehicleAgent):
        return {
            "color": VEC_STATION_COLORS[agent.station.unique_id],
            "size": 50,
        }

    # Render VEC station agent
    if isinstance(agent, VECStationAgent):
        portrayal = {
            "shape": "rect",
            "color": VEC_STATION_COLORS[agent.unique_id],
            "filled": "true",
            "layer": 0,
            "w": 1,
            "h": 1,
        }
        return portrayal

    assert False

In [3]:
import solara
from matplotlib.figure import Figure
from matplotlib.patches import Rectangle, Circle


def render_model(model):
    fig = Figure()
    ax = fig.subplots()

    min_x = min(waypoint[0] for waypoint in model.waypoints)
    max_x = max(waypoint[0] for waypoint in model.waypoints)
    min_y = min(waypoint[1] for waypoint in model.waypoints)
    max_y = max(waypoint[1] for waypoint in model.waypoints)
    road_width = max_x - min_x
    road_height = max_y - min_y

    # Draw the road
    road = Rectangle((min_x, min_y), road_width, road_height, linewidth=3, edgecolor='gray', facecolor='none')
    ax.add_patch(road)

    # Draw all agents
    for agent in model.agents:
        if isinstance(agent, VehicleAgent):
            ax.add_patch(Circle(agent.pos, 3, facecolor=VEC_STATION_COLORS[agent.station.unique_id], edgecolor='black'))
        elif isinstance(agent, VECStationAgent):
            color = VEC_STATION_COLORS[agent.unique_id]
            ax.add_patch(Rectangle((agent.pos[0] - 3, agent.pos[1] - 3), 6, 6, facecolor=color))
            range_circle = Circle(agent.pos, agent.range, color=color, fill=False, linestyle='--')
            ax.add_patch(range_circle)
            range_circle = Circle(agent.pos, agent.threshold * agent.range, color=color, fill=False, linestyle='--', alpha=0.5)
            ax.add_patch(range_circle)

    ax.set_xlim(0, model.width)
    ax.set_ylim(0, model.height)

    solara.FigureMatplotlib(fig)

In [4]:
page = JupyterViz(
    VECModel,
    model_params={"width": 100, "height": 80, "speed": 40},
    measures=[render_model, render_distance_chart],
    name="VEC Model",
    agent_portrayal=agent_portrayal,
)

# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text